In [1]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

#Loading data
got_dataframe = pd.read_csv("../input/character-predictions.csv", sep=",",na_values=[])
got_dataframe.fillna(0, inplace=True)

got_dataframe = got_dataframe.reindex(np.random.permutation(got_dataframe.index))
got_dataframe = got_dataframe[['title','male','culture','mother','father','house','spouse','book1','book2','book3','book4','book5','isMarried','isNoble','age','numDeadRelations','popularity','isPopular','isAlive']]
display.display(got_dataframe.describe())
got_dataframe.head()

,male,book1,book2,book3,book4,book5,isMarried,isNoble,age,numDeadRelations,popularity,isPopular,isAlive
count,1946.0,1946.0,1946.0,1946.0,1946.0,1946.0,1946.0,1946.0,1946.0,1946.0,1946.0,1946.0,1946.0
mean,0.6,0.2,0.4,0.5,0.6,0.4,0.1,0.5,-287.8,0.3,0.1,0.1,0.7
std,0.5,0.4,0.5,0.5,0.5,0.5,0.3,0.5,9236.0,1.4,0.2,0.2,0.4
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-298001.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
75%,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.1,0.0,1.0
max,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,100.0,15.0,1.0,1.0,1.0


,title,male,culture,mother,father,house,spouse,book1,book2,book3,book4,book5,isMarried,isNoble,age,numDeadRelations,popularity,isPopular,isAlive
1606,Lord of the Hornwood,1,Northmen,0,0,House Bolton of the Dreadfort,Donella Hornwood,0,1,1,1,1,1,1,23.0,1,0.5,1,1
1291,Lord Commander of the Night's Watch,1,0,0,0,House Qorgyle,0,0,0,1,0,0,0,1,0.0,0,0.1,0,0
1524,Rain House,1,0,0,0,House Wylde,0,0,0,0,0,0,0,1,0.0,0,0.3,0,0
1879,0,1,Northmen,0,0,House Glover,Sybelle Glover,1,1,1,1,1,1,0,0.0,0,0.2,0,1
1404,Ser,0,0,0,0,House Sunglass,0,0,0,1,1,1,0,1,0.0,0,0.0,0,1


In [2]:
def normalizeColumnNames(dataframe):
    dataframe.columns = [x.replace("'", "") for x in dataframe.columns]
    dataframe.columns = [x.replace(" ", "") for x in dataframe.columns]
    return dataframe

In [3]:
def preprocess_features(got_dataframe):
  selected_features = pd.concat([got_dataframe[['popularity','male','book1','book2','book3','book4','book5','isMarried','isNoble']],got_dataframe[[col for col in got_dataframe if col.startswith('norm_house')]] ],axis=1)
  processed_features = selected_features.copy()
  return processed_features

def preprocess_targets(got_dataframe):
  output_targets = pd.DataFrame()
  # Scale the target to be in units of thousands of dollars.
  output_targets["isAlive"] =  got_dataframe["isAlive"]
  return output_targets

In [4]:
preprocess_features(got_dataframe.head(1200))

,popularity,male,book1,book2,book3,book4,book5,isMarried,isNoble
1606,0.5,1,0,1,1,1,1,1,1
1291,0.1,1,0,0,1,0,0,0,1
1524,0.3,1,0,0,0,0,0,0,1
1879,0.2,1,1,1,1,1,1,1,0
1404,0.0,0,0,0,1,1,1,0,1
...,...,...,...,...,...,...,...,...,...
135,0.0,0,0,0,1,0,0,0,0
50,1.0,1,0,0,0,0,0,0,0
1853,0.7,1,1,1,1,1,1,0,0
1905,0.0,1,1,0,0,0,0,0,0


In [5]:
training_examples = preprocess_features(pd.concat([got_dataframe.head(1200),normalizeColumnNames(pd.get_dummies(got_dataframe[['house']],sparse=False, prefix=['norm_house'])).head(1200)], axis=1))
training_targets = preprocess_targets(got_dataframe.head(1200))

validation_examples = preprocess_features(pd.concat([got_dataframe.tail(746),normalizeColumnNames(pd.get_dummies(got_dataframe[['house']],sparse=False, prefix=['norm_house'])).tail(746)], axis=1))
validation_targets = preprocess_targets(got_dataframe.tail(746))

# Double-check that we've done the right thing.
print("Training examples summary:")
display.display(training_examples.describe())
print("Validation examples summary:")
display.display(validation_examples.describe())

print("Training targets summary:")
display.display(training_targets.describe())
print("Validation targets summary:")
display.display(validation_targets.describe())

Training examples summary:


,popularity,male,book1,book2,book3,book4,book5,isMarried,isNoble,norm_house_0,norm_house_AlchemistsGuild,norm_house_AntlerMen,norm_house_BandofNine,norm_house_BlackEars,norm_house_Blacks,norm_house_BraveCompanions,norm_house_BrotherhoodWithoutBanners,norm_house_BrotherhoodwithoutBanners,norm_house_Brotherhoodwithoutbanners,norm_house_BurnedMen,norm_house_Chatayasbrothel,norm_house_Citadel,norm_house_CityWatchofKingsLanding,norm_house_CompanyoftheCat,norm_house_Drownedmen,norm_house_FacelessMen,norm_house_FaithoftheSeven,norm_house_GoldenCompany,norm_house_GoodMasters,norm_house_Graces,norm_house_HappyPort,norm_house_HouseAllyrion,norm_house_HouseAmbrose,norm_house_HouseArryn,norm_house_HouseAshford,norm_house_HouseBaelish,norm_house_HouseBall,norm_house_HouseBanefort,norm_house_HouseBarEmmon,norm_house_HouseBaratheon,...,norm_house_HouseWynch,norm_house_HouseWythers,norm_house_HouseYarwyck,norm_house_HouseYew,norm_house_HouseYronwood,norm_house_HouseofGalare,norm_house_HouseofGhazeen,norm_house_HouseofKandaq,norm_house_HouseofLoraq,norm_house_HouseofMerreq,norm_house_HouseofPahl,norm_house_HouseofReznak,norm_house_IronBankofBraavos,norm_house_Khal,norm_house_KingdomoftheThreeDaughters,norm_house_Kingsguard,norm_house_KingswoodBrotherhood,norm_house_Maesters,norm_house_ManceRayder,norm_house_MoonBrothers,norm_house_NightsWatch,norm_house_Peach,norm_house_Pureborn,norm_house_Queensguard,norm_house_Rhllor,norm_house_Seawatch,norm_house_SecondSons,norm_house_StoneCrows,norm_house_Stormcrows,norm_house_SummerIslands,norm_house_TheCitadel,norm_house_Thenn,norm_house_Thirteen,norm_house_Three-eyedcrow,norm_house_UndyingOnes,norm_house_Unsullied,norm_house_Windblown,norm_house_WiseMasters,norm_house_brotherhoodwithoutbanners,norm_house_wildling
count,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,...,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0
mean,0.1,0.6,0.2,0.4,0.5,0.6,0.4,0.1,0.5,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.2,0.5,0.4,0.5,0.5,0.5,0.5,0.3,0.5,0.4,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.1,1.0,0.0,1.0,1.

Validation examples summary:


,popularity,male,book1,book2,book3,book4,book5,isMarried,isNoble,norm_house_0,norm_house_AlchemistsGuild,norm_house_AntlerMen,norm_house_BandofNine,norm_house_BlackEars,norm_house_Blacks,norm_house_BraveCompanions,norm_house_BrotherhoodWithoutBanners,norm_house_BrotherhoodwithoutBanners,norm_house_Brotherhoodwithoutbanners,norm_house_BurnedMen,norm_house_Chatayasbrothel,norm_house_Citadel,norm_house_CityWatchofKingsLanding,norm_house_CompanyoftheCat,norm_house_Drownedmen,norm_house_FacelessMen,norm_house_FaithoftheSeven,norm_house_GoldenCompany,norm_house_GoodMasters,norm_house_Graces,norm_house_HappyPort,norm_house_HouseAllyrion,norm_house_HouseAmbrose,norm_house_HouseArryn,norm_house_HouseAshford,norm_house_HouseBaelish,norm_house_HouseBall,norm_house_HouseBanefort,norm_house_HouseBarEmmon,norm_house_HouseBaratheon,...,norm_house_HouseWynch,norm_house_HouseWythers,norm_house_HouseYarwyck,norm_house_HouseYew,norm_house_HouseYronwood,norm_house_HouseofGalare,norm_house_HouseofGhazeen,norm_house_HouseofKandaq,norm_house_HouseofLoraq,norm_house_HouseofMerreq,norm_house_HouseofPahl,norm_house_HouseofReznak,norm_house_IronBankofBraavos,norm_house_Khal,norm_house_KingdomoftheThreeDaughters,norm_house_Kingsguard,norm_house_KingswoodBrotherhood,norm_house_Maesters,norm_house_ManceRayder,norm_house_MoonBrothers,norm_house_NightsWatch,norm_house_Peach,norm_house_Pureborn,norm_house_Queensguard,norm_house_Rhllor,norm_house_Seawatch,norm_house_SecondSons,norm_house_StoneCrows,norm_house_Stormcrows,norm_house_SummerIslands,norm_house_TheCitadel,norm_house_Thenn,norm_house_Thirteen,norm_house_Three-eyedcrow,norm_house_UndyingOnes,norm_house_Unsullied,norm_house_Windblown,norm_house_WiseMasters,norm_house_brotherhoodwithoutbanners,norm_house_wildling
count,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,...,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0,746.0
mean,0.1,0.6,0.2,0.4,0.5,0.6,0.4,0.1,0.5,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.2,0.5,0.4,0.5,0.5,0.5,0.5,0.4,0.5,0.4,0.1,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.1,0.0,0.1,0.1,0.1,0.1,0.0,0.0,0.0,0.1,...,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.1,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

Training targets summary:


,isAlive
count,1200.0
mean,0.7
std,0.4
min,0.0
25%,0.0
50%,1.0
75%,1.0
max,1.0


Validation targets summary:


,isAlive
count,746.0
mean,0.8
std,0.4
min,0.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


In [6]:
def construct_feature_columns(input_features):
  return set([tf.feature_column.numeric_column(my_feature)
              for my_feature in input_features])

def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a neural net regression model.
  
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
    
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                             
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
      ds = ds.shuffle(10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

In [7]:
training_examples.describe()

,popularity,male,book1,book2,book3,book4,book5,isMarried,isNoble,norm_house_0,norm_house_AlchemistsGuild,norm_house_AntlerMen,norm_house_BandofNine,norm_house_BlackEars,norm_house_Blacks,norm_house_BraveCompanions,norm_house_BrotherhoodWithoutBanners,norm_house_BrotherhoodwithoutBanners,norm_house_Brotherhoodwithoutbanners,norm_house_BurnedMen,norm_house_Chatayasbrothel,norm_house_Citadel,norm_house_CityWatchofKingsLanding,norm_house_CompanyoftheCat,norm_house_Drownedmen,norm_house_FacelessMen,norm_house_FaithoftheSeven,norm_house_GoldenCompany,norm_house_GoodMasters,norm_house_Graces,norm_house_HappyPort,norm_house_HouseAllyrion,norm_house_HouseAmbrose,norm_house_HouseArryn,norm_house_HouseAshford,norm_house_HouseBaelish,norm_house_HouseBall,norm_house_HouseBanefort,norm_house_HouseBarEmmon,norm_house_HouseBaratheon,...,norm_house_HouseWynch,norm_house_HouseWythers,norm_house_HouseYarwyck,norm_house_HouseYew,norm_house_HouseYronwood,norm_house_HouseofGalare,norm_house_HouseofGhazeen,norm_house_HouseofKandaq,norm_house_HouseofLoraq,norm_house_HouseofMerreq,norm_house_HouseofPahl,norm_house_HouseofReznak,norm_house_IronBankofBraavos,norm_house_Khal,norm_house_KingdomoftheThreeDaughters,norm_house_Kingsguard,norm_house_KingswoodBrotherhood,norm_house_Maesters,norm_house_ManceRayder,norm_house_MoonBrothers,norm_house_NightsWatch,norm_house_Peach,norm_house_Pureborn,norm_house_Queensguard,norm_house_Rhllor,norm_house_Seawatch,norm_house_SecondSons,norm_house_StoneCrows,norm_house_Stormcrows,norm_house_SummerIslands,norm_house_TheCitadel,norm_house_Thenn,norm_house_Thirteen,norm_house_Three-eyedcrow,norm_house_UndyingOnes,norm_house_Unsullied,norm_house_Windblown,norm_house_WiseMasters,norm_house_brotherhoodwithoutbanners,norm_house_wildling
count,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,...,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0,1200.0
mean,0.1,0.6,0.2,0.4,0.5,0.6,0.4,0.1,0.5,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
std,0.2,0.5,0.4,0.5,0.5,0.5,0.5,0.3,0.5,0.4,0.0,0.0,0.0,0.0,0.0,0.1,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.1,0.0,0.0,0.0,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25%,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50%,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75%,0.1,1.0,0.0,1.0,1.

In [8]:
dict(training_examples).items()

dict_items([('popularity', 1606   0.5
1291   0.1
1524   0.3
1879   0.2
1404   0.0
        ..
135    0.0
50     1.0
1853   0.7
1905   0.0
383    0.0
Name: popularity, Length: 1200, dtype: float64), ('male', 1606    1
1291    1
1524    1
1879    1
1404    0
       ..
135     0
50      1
1853    1
1905    1
383     1
Name: male, Length: 1200, dtype: int64), ('book1', 1606    0
1291    0
1524    0
1879    1
1404    0
       ..
135     0
50      0
1853    1
1905    1
383     0
Name: book1, Length: 1200, dtype: int64), ('book2', 1606    1
1291    0
1524    0
1879    1
1404    0
       ..
135     0
50      0
1853    1
1905    0
383     0
Name: book2, Length: 1200, dtype: int64), ('book3', 1606    1
1291    1
1524    0
1879    1
1404    1
       ..
135     1
50      0
1853    1
1905    0
383     0
Name: book3, Length: 1200, dtype: int64), ('book4', 1606    1
1291    0
1524    0
1879    1
1404    1
       ..
135     0
50      0
1853    1
1905    0
383     0
Name: book4, Length: 1200, dtype: int

In [9]:
def train_nn_regression_model(
    learning_rate,
    steps,
    batch_size,
    hidden_units,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets,
    targetLabel="isAlive"):
  """Trains a neural network regression model.
  
  In addition to training, this function also prints training progress information,
  as well as a plot of the training and validation loss over time.
  
  Args:
    learning_rate: A `float`, the learning rate.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    hidden_units: A `list` of int values, specifying the number of neurons in each layer.
    training_examples: A `DataFrame` containing one or more columns from
      `california_housing_dataframe` to use as input features for training.
    training_targets: A `DataFrame` containing exactly one column from
      `california_housing_dataframe` to use as target for training.
    validation_examples: A `DataFrame` containing one or more columns from
      `california_housing_dataframe` to use as input features for validation.
    validation_targets: A `DataFrame` containing exactly one column from
      `california_housing_dataframe` to use as target for validation.
    targetLabel: A string, represent the target label
      
  Returns:
    A `DNNRegressor` object trained on the training data.
  """

  periods = 10
  steps_per_period = steps / periods
  
  # Create a DNNRegressor object.
  my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  dnn_regressor = tf.estimator.DNNRegressor(
      feature_columns=construct_feature_columns(training_examples),
      hidden_units=hidden_units,
      optimizer=my_optimizer,
  )
  
  # Create input functions.
  training_input_fn = lambda: my_input_fn(training_examples, 
                                          training_targets["isAlive"], 
                                          batch_size=batch_size)
  predict_training_input_fn = lambda: my_input_fn(training_examples, 
                                                  training_targets["isAlive"], 
                                                  num_epochs=1, 
                                                  shuffle=False)
  predict_validation_input_fn = lambda: my_input_fn(validation_examples, 
                                                    validation_targets["isAlive"], 
                                                    num_epochs=1, 
                                                    shuffle=False)

  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print("Training model...")
  print("RMSE (on training data):")
  training_rmse = []
  validation_rmse = []
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    dnn_regressor.train(
        input_fn=training_input_fn,
        steps=steps_per_period
    )
    # Take a break and compute predictions.
    training_predictions = dnn_regressor.predict(input_fn=predict_training_input_fn)
    training_predictions = np.array([item['predictions'][0] for item in training_predictions])
    
    validation_predictions = dnn_regressor.predict(input_fn=predict_validation_input_fn)
    validation_predictions = np.array([item['predictions'][0] for item in validation_predictions])
    
    # Compute training and validation loss.
    training_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(training_predictions, training_targets))
    validation_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(validation_predictions, validation_targets))
    # Occasionally print the current loss.
    print("  period %02d : %0.2f" % (period, training_root_mean_squared_error))
    # Add the loss metrics from this period to our list.
    training_rmse.append(training_root_mean_squared_error)
    validation_rmse.append(validation_root_mean_squared_error)
  print("Model training finished.")

  # Output a graph of loss metrics over periods.
  plt.ylabel("RMSE")
  plt.xlabel("Periods")
  plt.title("Root Mean Squared Error vs. Periods")
  plt.tight_layout()
  plt.plot(training_rmse, label="training")
  plt.plot(validation_rmse, label="validation")
  plt.legend()

  print("Final RMSE (on training data):   %0.2f" % training_root_mean_squared_error)
  print("Final RMSE (on validation data): %0.2f" % validation_root_mean_squared_error)

  return dnn_regressor

In [10]:
#_ = training_examples.hist(bins=20, figsize=(18, 12), xlabelsize=10)
print(len(training_examples.columns))
print("-----")
print(len(training_targets.columns))
print("-----")
print(len(training_examples.columns))
print("-----")
print(len(validation_targets.columns))

357
-----
1
-----
357
-----
1


In [ ]:
dnn_regressor = train_nn_regression_model(
    learning_rate=0.005,
    steps=1000,
    batch_size=10,
    hidden_units=[2,1],
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Training model...
RMSE (on training data):
  period 00 : 0.44
  period 01 : 0.44
  period 02 : 0.44
  period 03 : 0.43
  period 04 : 0.43
  period 05 : 0.42
  period 06 : 0.41
  period 07 : 0.41
  period 08 : 0.40
